In [1]:
import os

In [2]:
import sys
sys.path.append('../')
from src import *

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
model = Load_model(resnet_val=50, device = device)

save_folder_addr = '../res50/'
model_file_name = 'Eurosat50'
model.load_state_dict(torch.load(save_folder_addr + model_file_name))

Model Loaded


<All keys matched successfully>

In [5]:
data_folder = 'C:/dev/EuroSAT/'
root = os.path.join(data_folder, '2750/')

In [6]:
if __name__ == '__main__':
    torch.manual_seed(0)
    np.random.seed(0)
    torch.cuda.manual_seed(0)
    random.seed(0)
    
    print("pytorch version", torch.__version__)
    
    criteria = torch.nn.CrossEntropyLoss()
    resnet_model = Load_model(resnet_val=50, device = device)
    train_loader, val_loader = make_dataloaders(root, batch_size_val=128)
    
    resnet_model=resnet_model.to(device)

pytorch version 2.1.2
Model Loaded


In [7]:
len(train_loader.dataset), len(val_loader.dataset)

(24300, 2700)

In [8]:
import hashlib

val_data_string = str(val_loader.dataset)
current_hash = hashlib.sha256(val_data_string.encode()).hexdigest()

with open(save_folder_addr + 'val_data_hash.txt', 'r') as file:
    original_hash = file.readline()


In [9]:
original_hash

'71aed80bf63e52ddddbcba97c1eb0277ec019597ee778a114c8a265b8bdb2283'

In [10]:
current_hash

'71aed80bf63e52ddddbcba97c1eb0277ec019597ee778a114c8a265b8bdb2283'

In [11]:
if current_hash == original_hash:
    print('Validaiton set is the same')
else:
    Exception('Validation Set is different')    

Validaiton set is the same


In [20]:
from sklearn.metrics import confusion_matrix

model.eval()

all_labels = []
all_predictions = []

for inputs, labels in val_loader:
    inputs = inputs.to(device)
    labels = labels.to(device)

    with torch.no_grad():
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

    all_labels.extend(labels.cpu().numpy())
    all_predictions.extend(preds.cpu().numpy())

cm = confusion_matrix(all_labels, all_predictions)
print(cm)


[[251   0  20   1   0   3  16   0   3   5]
 [  0 270   3   0   0   1   1   0   0  24]
 [  2   0 253   0   0   1  41   0   0   1]
 [  0   0  11 233   2   0   1   1   1   1]
 [  0   0  40   3 205   0   0   0   0   0]
 [  4  22  40   3   0 119   6   0   3   3]
 [  2   0  24   1   0   0 221   0   0   0]
 [  0   0 109   3  18   0   5 163   0   0]
 [ 11   0  14  16   1   1   2   0 203   2]
 [  1   1   2   2   0   1   0   0   6 285]]


In [16]:
print(cm)


[[251   0  20   1   0   3  16   0   2   5]
 [  0 270   3   0   0   1   1   0   0  24]
 [  2   0 255   0   0   1  40   0   0   1]
 [  0   0  11 231   2   0   1   1   1   1]
 [  0   0  40   3 205   0   0   0   0   0]
 [  4  22  40   3   0 119   6   0   3   3]
 [  2   0  26   1   0   0 220   0   0   0]
 [  0   0 109   3  18   0   5 165   0   0]
 [ 11   0  14  16   1   1   2   0 201   2]
 [  1   1   2   2   0   1   0   0   6 286]]


In [13]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

class_labels = [name for name in os.listdir(root) if os.path.isdir(os.path.join(root, name))]

cm_sum = np.sum(cm, axis=1, keepdims=True)
cm_norm = cm / cm_sum.astype(float) * 100

plt.figure(figsize=(20, 20))
sns.heatmap(cm_norm, annot=True, fmt=".2f", cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.title("Normalized Confusion Matrix")
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()